# 0 - Imports

In [38]:
# Manejo de datos
import pandas as pd
import numpy as np
import os

# Visualización de datos
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Red Neuronal
from tensorflow.keras import models
from tensorflow.keras import layers
import kerastuner as kt
import tensorflow as tf

# MLFlow
import mlflow

# Utilidades
import sys
sys.path.append("..\..")
from package_utilities import mlflow_utils as mlf
from joblib import dump # Gurdado local modelo
import IPython

# 1- Carga de datos

In [39]:
X_train = pd.read_parquet("../../data/train_set_test/TfidfVectorizer/X_train.parquet")
X_val = pd.read_parquet("../../data/train_set_test/TfidfVectorizer/X_val.parquet")
Y_train = pd.read_parquet("../../data/train_set_test/TfidfVectorizer/Y_train.parquet")
Y_val = pd.read_parquet("../../data/train_set_test/TfidfVectorizer/Y_val.parquet")

# 2 - Construcción de la red neuronal artificial

## Hypertuning

In [40]:
def model_builder(hp):
    # Definición del modelo
    model = keras.Sequential()

    # Tuning del número de neuronas de las hidden layer
    hp_units = hp.Int('units', min_value=8, max_value=128, step=8)
    
    # Tuning del número de capas
    hp_layers = hp.Int('layers', min_value=1, max_value=8, step=1)
    
    # Input layer
    model.add(layers.Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)))
    
    # Hidden layers 
    for i in range(hp_layers):
        # Añadimos la hidden layer
        model.add(layers.Dense(units=hp_units, activation='relu'))
        
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Seleccionamos el valor optimo entre [0.01, 0.001, 0.0001]
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
        loss='categorical_crossentropy',
        metrics=['accuracy', 'Precision']
    )
    
    return model

In [46]:
tuner = kt.Hyperband(
    model_builder,
    objective= 'val_accuracy',
    max_epochs=20,
    factor=5,
    directory='hp_dir',
    project_name='ruters_dataset_2'
)

In [47]:
# Creamos una función de callback que limpie el output
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [48]:
# Ejecución del proceso de tuning
tuner.search(
    X_train, 
    Y_train,
    epochs=10,
    validation_data=(X_val, Y_val),
    callbacks=[ClearTrainingOutput()]
)

Trial 13 Complete [00h 01m 57s]
val_accuracy: 0.5700525641441345

Best val_accuracy So Far: 0.5700525641441345
Total elapsed time: 00h 13m 11s


In [49]:
# Obtención de los hiperparámetros óptimos
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

{'units': 120,
 'layers': 7,
 'learning_rate': 0.01,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [50]:
print("Units:", best_hps.get('units'))
print("Layers:", best_hps.get('layers'))

Units: 120
Layers: 7
